In [1]:
# from IPython.display import display
# from PIL import Image, ImageDraw

In [2]:
import basic_entities
from glob import glob
import scipy.io

In [3]:
# frames = glob('../data/ucsdpeds/images/vidf/vidf1_*/*')
# frames.sort()
# len(frames)

In [4]:
# frames_orig = glob('../data/ucsdpeds/orig_images/out_*.png')
# frames_orig.sort()
# len(frames_orig)

In [5]:
# mat = scipy.io.loadmat('../data/ucsdpeds/line_crossing/lineInfo.mat')
# for i in mat['lineInfo'][0][0]:
#     print(i)

In [6]:
# factor = Image.open(frames_orig[0]).size[0] / (Image.open(frames[0]).size[0])
# print(factor)

In [7]:
# mat = scipy.io.loadmat('../data/ucsdpeds/line_crossing/cgt_s_all.mat')
# left_cross = mat['cgt_s_all'][0][0][0] # Left
# right_cross = mat['cgt_s_all'][0][1][0] # Right

In [8]:
# for i, v in enumerate(mat['cgt_s_all'][0][0][0]):
#     if v > 0:
#         img = Image.open(frames_orig[i])
#         draw = ImageDraw.Draw(img) 3 
#         draw.line((100*factor,1*factor, 100*factor,158*factor), fill=255, width=5)
#         display(img)
#         break

In [9]:
# path = '../data/ucsdpeds'
# small_videos = glob('{}/images/vidf/vidf1_*/'.format(path))
# small_videos.sort()
# small_videos

# location_mats = glob('{}/gt/vidf/vidf1_33_*_frame_full.mat'.format(path))
# location_mats.sort()

# frames_orig = glob('{}/orig_images/out_*.png'.format(path))
# frames_orig.sort()

# all_small_frames = glob('{}/images/vidf/vidf1_*/*'.format(path))
# all_small_frames.sort()

In [15]:
import scipy.io
import basic_entities
from glob import glob

def get_point_locations_mat(path, factor=3.0252):
    mat = scipy.io.loadmat(path)
    frames = []
    for i, frame in enumerate(mat['fgt'][0][0][0][0]):
        frame_ret = []
        dots = frame[0][0][0] # (X, Y, 1)
        for dot in dots:
            x = dot[0] * factor
            y = dot[1] * factor
            frame_ret.append([x, y])
        
        frames.append(frame_ret)
    return frames

def load_videos(path, factor=3.0252):
    mat = scipy.io.loadmat('{}/line_crossing/cgt_s_all.mat'.format(path))
    left_cross = mat['cgt_s_all'][0][0][0] # Left
    right_cross = mat['cgt_s_all'][0][1][0] # Right
    
    small_videos = glob('{}/images/vidf/vidf1_*/'.format(path))
    small_videos.sort()
    small_videos

    location_mats = glob('{}/gt/vidf/vidf1_33_*_frame_full.mat'.format(path))
    location_mats.sort()

    frames_orig = glob('{}/orig_images/out_*.png'.format(path))
    frames_orig.sort()

    all_small_frames = glob('{}/images/vidf/vidf1_*/*'.format(path))
    all_small_frames.sort()
    
    videos = []
    
    for small_video, location_mat in zip(small_videos, location_mats):
        points = get_point_locations_mat(location_mat, factor)
        small_frames = glob('{}*.png'.format(small_video))
        small_frames.sort()

        first_frame_id = all_small_frames.index(small_frames[0])
        last_frame_id = all_small_frames.index(small_frames[-1])

        video = basic_entities.BasicVideo('{}_{}'.format(first_frame_id, last_frame_id))

        if first_frame_id < 2000:
            line = basic_entities.BasicLineSample(small_video, (100*factor,1*factor), (100*factor,158*factor))
            line.set_crossed(sum(left_cross[first_frame_id:last_frame_id+1]),
                            sum(right_cross[first_frame_id:last_frame_id+1]))
            video.add_line(line)

        for dots, s_frame in zip(points, small_frames):
            o_frame = frames_orig[all_small_frames.index(s_frame)]
            frame = basic_entities.BasicFrame(o_frame)
            for dot in dots:
                frame.add_point(dot)

            video.add_frame(frame)
            
        videos.append(video)

    return videos

In [16]:
# for small_video, location_mat in zip(small_videos, location_mats):
#     points = get_point_locations_mat(location_mat)
#     small_frames = glob('{}*.png'.format(small_video))
#     small_frames.sort()
    
#     first_frame_id = all_small_frames.index(small_frames[0])
#     last_frame_id = all_small_frames.index(small_frames[-1])
        
#     video = basic_entities.BasicVideo(small_video)
    
#     if first_frame_id < 2000:
#         line = basic_entities.BasicLineSample(small_video, (100*factor,1*factor), (100*factor,158*factor))
#         line.set_crossed(sum(left_cross[first_frame_id:last_frame_id+1]),
#                         sum(right_cross[first_frame_id:last_frame_id+1]))
#         video.add_line(line)
    
#     for dots, s_frame in zip(points, small_frames):
#         o_frame = frames_orig[all_small_frames.index(s_frame)]
#         frame = basic_entities.BasicFrame(o_frame)
#         for dot in dots:
#             frame.add_point(dot)
        
#         video.add_frame(frame)
        
#     if len(video.get_lines()) > 0:
#         print(video.get_lines()[0].get_crossed())

In [17]:
videos = load_videos('../data/ucsdpeds')

In [23]:
for i, video in enumerate(videos):
    print(i, video.get_path())

0 0_199
1 200_399
2 400_599
3 600_799
4 800_999
5 1000_1199
6 1200_1399
7 1400_1599
8 1600_1799
9 1800_1999
10 2000_2199
11 2200_2399
12 2400_2599
13 2600_2799
14 2800_2999
15 3000_3199
16 3200_3399
17 3401_3600
18 3601_3800
19 3801_4000
